
# Environment Setup


In [3]:
#!pip install transformers
#pip install datasets
#!pip install transformers[torch]
#!pip install accelerate>=0.20.1
#!pip install evaluate

In [45]:
import os
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer,TrainingArguments, pipeline, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from datasets import load_dataset
import time
import warnings
warnings.simplefilter("ignore")

# Prepare Dataset


In [56]:
dataset = load_dataset("csv", data_files="Data_with_explanations.csv")

In [58]:
dataset=pd.read_csv("Data_with_explanations.csv")
dataset

,question,response,explanation
0,What is the purpose of clicking on the 'Ajoute...,To fill in the characteristics of Finition,"The purpose of clicking the ""Ajouter Finition""..."
1,What types of information can be filled in whe...,"Caractéristiques, Motorisation, Transmission e...",In the context of automotive design and functi...
2,What is the first category of information that...,Caractéristiques,"When clicking on ""Ajouter fin"", I will explain..."
3,Is there a separate section for motorization-r...,"Yes, it's called Motorisation","Yes, the provided sections are separated into ..."
4,What is the last category of information that ...,Equipements,"The ""last category of information that can be ..."
...,...,...,...
745,What are the steps to follow to choose a product?,The client must follow the steps by choosing t...,"To choose a product, the following steps can b..."
746,What is required of the client at the beginning?,The client must choose the product according t...,The client must first select from the offered ...
747,What action does the client need to take first?,Clicking on 'Financer'.,Sure! Let's break down how the client needs to...
748,How can the client proceed with selecting a pr...,By choosing the product according to the offer...,Explanation:\n\nThe client should follow these...


In [ ]:
!pip install evaluate

## Tokenizer

In [ ]:
student_checkpoint = "gpt2"
from transformers import AutoTokenizer, AutoModelForCausalLM
student_tokenizer = AutoTokenizer.from_pretrained(student_checkpoint)
llm_model = AutoModelForCausalLM.from_pretrained(student_checkpoint).to(device)

model.safetensors:  15%|#5        | 83.9M/548M [00:00<?, ?B/s]

In [ ]:
teacher_checkpoint = "gpt2-xl"
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_checkpoint)
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_checkpoint).to("cuda" if torch.cuda.is_available() else "cpu")


In [34]:
def tokenize_text(batch):
  return student_tokenizer(batch["question"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

# Prepare training

Create trainer class and loss function compute_loss

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [39]:
class KnowledgeDistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    super().__init__(*args, **kwargs)
    self.alpha = alpha
    self.temperature = temperature

In [40]:
class KnowledgeDistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self, model, inputs, return_outputs=False):
    #Extract cross-entropy loss and logits from student
    outputs_student = model(**inputs)
    loss_ce = outputs_student.loss
    logits_student = outputs_student.logits
    # Extract logits from teacher
    outputs_teacher = self.teacher_model(**inputs)
    logits_teacher = outputs_teacher.logits
     #Computing distillation loss by Softening probabilities
    loss_fct = nn.KLDivLoss(reduction="batchmean")
    loss_kd = self.args.temperature ** 2 * loss_fct(
                F.log_softmax(logits_student / self.args.temperature, dim=-1),
                F.softmax(logits_teacher / self.args.temperature, dim=-1))

    loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
    return (loss, outputs_student) if return_outputs else loss

In [46]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)


In [50]:
batch_size = 48
finetuned_student_ckpt = "student_aziz"

## Training Arguments for DistillationTrainer
student_training_args = KnowledgeDistillationTrainingArguments(
    output_dir=finetuned_student_ckpt, 
    evaluation_strategy = "epoch",
    num_train_epochs=3, 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, 
    alpha=1, 
    weight_decay=0.01)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [52]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

ERROR: Invalid requirement: "'accelerate": Expected package name at the start of dependency specifier
    'accelerate
    ^


In [ ]:


%%time
distilbert_trainer = KnowledgeDistillationTrainer(
    model_init=student_init,
    teacher_model=teacher_model, 
    args=student_training_args,
    train_dataset=clinc_tokenized['train'], 
    eval_dataset=clinc_tokenized['validation'],
    compute_metrics=compute_metrics, 
    tokenizer=student_tokenizer)

distilbert_trainer.train()

